In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
import _pickle as pickle
from typing import List
import os
from collections import defaultdict
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import json_rec
from data_import import load_embedding_from_disks

In [3]:
df = pd.read_csv("./data/text_emotion.csv")

In [4]:
if not os.path.exists("./saved_model"):
    os.mkdir("./saved_model")
now = datetime.datetime.now()
model_dir = "./saved_model/" + now.strftime("%Y-%m-%d-%H:%M")
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    os.mkdir(model_dir + "/tensorboard")

In [5]:
param_writer = json_rec.ParamWriter()

In [6]:
# Parameter
# GLOVE_FILENAME = "../data/glove.840B.300d.txt"
use_default = bool(input("Use default param? >>> ").upper() == "Y")
if use_default:
    param = dict(
        GLOVE_FILENAME="../data/glove.6B.50d.txt",
        lstm_units=(512, 1024),
        num_classes=13,
        epochs=10,
        max_seq_length=40
    )
else:
    config_file = input("Directory of config file >>> ")
    param_writer.read(config_file)

Use default param? >>> Y


In [7]:
json_rec.write_param(param, "./sample_config.json")

In [8]:
print("Loading embedding from disks...")
word_to_index, index_to_embedding = load_embedding_from_disks(
    param["GLOVE_FILENAME"],
    with_indexes=True
)
print("Embedding loaded from disks.")

10155it [00:00, 50912.39it/s]

Loading embedding from disks...


400000it [00:07, 51161.14it/s]


Embedding loaded from disks.


In [9]:
vocab_size, embedding_dim = index_to_embedding.shape
print(f"Embedding matrix loaded, Vocab Size: {vocab_size}\nEmbedding Dim: {embedding_dim}")
SENTI_LST = list(set(df["sentiment"]))
globals().update(param)

Embedding matrix loaded, Vocab Size: 400001
Embedding Dim: 50


In [10]:
def word2int(w: str) -> int:
    try:
        idx = word_to_index[w]
    except KeyError:
        idx = word_to_index["unk"]
    return idx

In [11]:
X_lst, y_lst = [], []
for sentence, senti in zip(df["content"], df["sentiment"]):
    # ==== Encode x ====
    tokens = sentence.lower().split()
    word_ints = np.array([word2int(x) for x in tokens])
    X_lst.append(word_ints)
    
    # ==== Encode y ====
    label = np.zeros([num_classes])
    senti_index = SENTI_LST.index(senti)
    label[senti_index] = 1
    y_lst.append(label)
    
X_lst = pad_sequences(
    X_lst,
    maxlen=max_seq_length,
    padding="post",
    truncating="post"
)

X_raw = np.stack(X_lst)
y_raw = np.stack(y_lst)
print(X_raw.shape)
print(y_raw.shape)

(40000, 40)
(40000, 13)


In [12]:
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X_raw, y_raw,
    test_size=0.2,
    shuffle=True
)

(X_train, X_val,
 y_train, y_val) = train_test_split(
    X_train, y_train,
    test_size=0.2,
    shuffle=True
)

In [13]:
print(f"Training and testing set generated,\
\nX_train shape: {X_train.shape}\
\ny_train shape: {y_train.shape}\
\nX_test shape: {X_test.shape}\
\ny_test shape: {y_test.shape}\
\nX_validation shape: {X_val.shape}\
\ny_validation shape: {y_val.shape}")

Training and testing set generated,
X_train shape: (25600, 40)
y_train shape: (25600, 13)
X_test shape: (8000, 40)
y_test shape: (8000, 13)
X_validation shape: (6400, 40)
y_validation shape: (6400, 13)


In [14]:
X_train_batches = X_train.reshape(25, 1024, max_seq_length)
y_train_batches = y_train.reshape(25, 1024, num_classes)

In [15]:
try:
    sess.close()
except NameError:
    print("Session already cleaned.")

Session already cleaned.


In [16]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

with tf.name_scope("DATA_IO"):
    word_ids = tf.placeholder(
        tf.int32,
        shape=[None, max_seq_length]
    )
    
    y = tf.placeholder(
        tf.float32,
        shape=[None, num_classes]
    )

with tf.name_scope("EMBEDDING"):
    embedding = tf.Variable(
        tf.constant(0.0, shape=index_to_embedding.shape),
        trainable=False,
        name="EMBEDDING"
    )
    
    word_representation_layer = tf.nn.embedding_lookup(
        params=embedding,
        ids=word_ids
    )
    
    embedding_placeholder = tf.placeholder(
        tf.float32,
        shape=index_to_embedding.shape
    )
    
    embedding_init = embedding.assign(embedding_placeholder)
    
    _ = sess.run(
        embedding_init, 
            feed_dict={
                embedding_placeholder: index_to_embedding
        }
    )

cell = tf.nn.rnn_cell.MultiRNNCell(
    [tf.nn.rnn_cell.LSTMCell(
        num_units=units,
        name=f"LSTM_LAYER_{i}")
        for i, units in enumerate(lstm_units)
     ])

lstm_cell = tf.contrib.rnn.DropoutWrapper(
    cell=cell,
    output_keep_prob=0.75
)
outputs, state = tf.nn.dynamic_rnn(
    lstm_cell, 
    word_representation_layer,
    dtype=tf.float32
)

with tf.name_scope("OUTPUT"):
    weight = tf.Variable(
        tf.truncated_normal(
            [lstm_units[-1], num_classes]
        )
    )
    
    bias = tf.Variable(
        tf.random_normal(shape=[num_classes])
    )

# Option i)
    value = tf.transpose(outputs, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
# Option ii)
#     last = outputs[:, -1, :]
    pred = tf.matmul(last, weight) + bias
    pred_idx = tf.argmax(pred, axis=1)

In [17]:
with tf.name_scope("METRICS"):
    correct_pred = tf.equal(
        tf.argmax(pred, axis=1),
        tf.argmax(y, axis=1)
    )

    accuracy = tf.reduce_mean(
        tf.cast(correct_pred, tf.float32)
    )

with tf.name_scope("LOSSES"):
    loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=pred,
            labels=y
        )
    )
    optimizer = tf.train.AdamOptimizer().minimize(loss)

tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<tf.Tensor 'Accuracy:0' shape=() dtype=string>

In [18]:
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
logdir = model_dir + "/tensorboard/"
writer = tf.summary.FileWriter(logdir, sess.graph)

for e in range(epochs):
    for X_batch, y_batch in zip(X_train_batches, y_train_batches):
        sess.run(
            optimizer,
            feed_dict={
                word_ids: X_batch,
                y: y_batch
            }
        )

    if e % 5 == 0:
        summary = sess.run(
            merged,
            feed_dict={
                word_ids: X_val,
                y: y_val
            }
        )
    if e % 1 == 0:
        train_acc = []
        for X_batch, y_batch in zip(X_train_batches, y_train_batches):
            train_acc.append(accuracy.eval(
                feed_dict={word_ids: X_batch, y: y_batch}
            ))
        avg_tarin_acc = np.mean(train_acc)
        val_acc = accuracy.eval(feed_dict={word_ids: X_val, y: y_val})
        print(
            f"Epochs[{e}]: train batch avg accuracy={avg_tarin_acc}, val accuracy={val_acc}")
    writer.add_summary(summary, e)
    writer.close()
f = lambda src: pred_idx.eval(feed_dict={word_ids: src})
# train_pred = f(X_train)
test_pred = f(X_test)
val_pred = f(X_val)

Epochs[0]: train batch avg accuracy=0.21257811784744263, val accuracy=0.21453124284744263
Epochs[1]: train batch avg accuracy=0.21019531786441803, val accuracy=0.21906250715255737
Epochs[2]: train batch avg accuracy=0.21074219048023224, val accuracy=0.21390624344348907
Epochs[3]: train batch avg accuracy=0.20976562798023224, val accuracy=0.21406249701976776
Epochs[4]: train batch avg accuracy=0.21320313215255737, val accuracy=0.21375000476837158
Epochs[5]: train batch avg accuracy=0.21277344226837158, val accuracy=0.21296875178813934
Epochs[6]: train batch avg accuracy=0.21269531548023224, val accuracy=0.22374999523162842
Epochs[7]: train batch avg accuracy=0.21125000715255737, val accuracy=0.2135937511920929
Epochs[8]: train batch avg accuracy=0.20972655713558197, val accuracy=0.21968750655651093
Epochs[9]: train batch avg accuracy=0.21160155534744263, val accuracy=0.21578125655651093


In [25]:
tf.saved_model.simple_save(
            sess,
            export_dir=f"{model_dir}/model",
            inputs={"word_ids": word_ids, "y": y},
            outputs={"pred": pred, "pred_idx": pred_idx})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/2019-01-20-01:38/model/saved_model.pb


In [ ]:
def sentence2ints(sentence):
    tokens = sentence.split()
    ids = [word_to_index[word] for word in tokens]
    ids = pad_sequences([ids], maxlen=max_seq_length, padding="post", truncating="post")
    return np.array(ids)